# Running Commands on Amazon Bedrock AgentCore Code Interpreter - Tutorial

This tutorial demonstrates how to use Amazon Bedrock AgentCore Code Interpreter to run commands (shell and AWS CLI). We will interact with AWS services, specifically focusing on S3 operations. We'll walk through:

1. Creating a code interpreter
2. Start code interpreter session
3. Run Commands(shell and AWS CLI)
5. Performing S3 operations(create bucket, copy objects, list bucket objects)
6. Cleanup (stop session and delete code interpreter)



## Prerequisites
- AWS account with Bedrock AgentCore Code Interpreter access
- You have the necessary IAM permissions to create and manage code interpreter resources
- YOu have the necessary IAM permissions to perform s3 operations
- Required Python packages installed(including boto3 & bedrock-agentcore)


## Your IAM execution role should have the following IAM policy attached

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}

#### The IAM execution role should also have the following trust policy attached. By including bedrock-agentcore.amazonaws.com in the trust policy, you're allowing the Bedrock Agent service itself to assume this IAM role and perform actions on your behalf(such as Amazon S3)

```{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::558228106740:root",
                "Service": [
                    "bedrock-agentcore.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole",
            "Condition": {}
        }
    ]
}

Additionally, you need to attach the AmazonS3FullAccess IAM policy to the IAM Execution role to perform the s3 operations described in this tutorial

## How it works

The code execution sandbox enables agents to safely process user queries by creating an isolated environment with a code interpreter, shell, and file system. After a Large Language Model helps with tool selection, code is executed within this session, before being returned to the user or Agent for synthesis.

![architecture local](code-interpreter.png)

## 1. Setting Up the Environment

First, let's import the necessary libraries

In [ ]:
!pip install --upgrade -r requirements.txt

In [1]:
import json
import boto3
from bedrock_agentcore.tools.code_interpreter_client import code_session, CodeInterpreter
from bedrock_agentcore._utils import endpoints
import time
from typing import Dict, Any, List

## 2. Configuration Variables

Set up the necessary configuration variables for code interpreter and S3 operations. We also provide an IAM execution role to pass to the code interpreter,so that it can assume it to access other AWS resources. This role needs S3 permissions, as discussed above

In [2]:
# Configuration setup
execution_role_arn = "arn:aws:iam::615299776985:role/agentcore-agentcore_strands-role"
unique_bucket_name = f"amzn-bucket-{int(time.time())}"
s3_path = f"s3://{unique_bucket_name}"

region = "us-west-2"

#local file that we will upload to Code interpreter and then to an S3 bucket
local_file = "samples/stats.py"

## 3. Setting up endpoints

We need to configure both data plane and control plane endpoints to create the boto3 clients.

In [3]:
# Configure endpoints
data_plane_endpoint = endpoints.get_data_plane_endpoint(region)
control_plane_endpoint = endpoints.get_control_plane_endpoint(region)

In [5]:
control_plane_endpoint

'https://bedrock-agentcore-control.us-west-2.amazonaws.com'

## 4. Creating AWS Clients

Initialize boto3 clients for both control plane and data plane operations.

In [4]:
# Create boto3 clients
cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=control_plane_endpoint)

dp_client = boto3.client("bedrock-agentcore", 
                        region_name=region,
                        endpoint_url=data_plane_endpoint)

## 5. Creating a Code Interpreter

Create a code interpreter instance with specific configuration parameters.

When configuring a code interpreter, you can choose network settings (Sandbox, Public, or VPC), dependencies configuration, security settings, and permissions through an IAM runtime role that defines what AWS resources the code interpreter can access.

In [22]:
# Create code interpreter
unique_name = f"s3InteractionEnv_{int(time.time())}"
interpreter_response = cp_client.create_code_interpreter(
    name=unique_name,
    description="Environment for S3 file operations",
    executionRoleArn=execution_role_arn,
    networkConfiguration={
        'networkMode': 'PUBLIC'
    }
)
interpreter_id = interpreter_response["codeInterpreterId"]
print(f"Created interpreter: {interpreter_id}")

Created interpreter: s3InteractionEnv_1756890754-TCHTCJfZGz


## 6. Starting a Session

Create a session within the code interpreter to execute code.

In [24]:
# Start session
session_response = dp_client.start_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    name="s3InteractionSession",
    sessionTimeoutSeconds=900
)
session_id = session_response["sessionId"]
print(f"Created session: {session_id}")

Created session: 01K47DK9RV3KYMKXYZ58ZE19GW


## 7. Helper Function for Tool Execution

Define a utility function to simplify code interpreter tool invocation

In [25]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    response = dp_client.invoke_code_interpreter(
        codeInterpreterIdentifier=interpreter_id,
        sessionId=session_id,
        name=tool_name,
        arguments=arguments
    )
    for event in response["stream"]:
        return json.dumps(event["result"], indent=2)

## 8. Test Code Execution

### 8.1 Test the code interpreter with a simple Hello World example.

In [26]:
# Test code execution
# S3 Operations
print("executing shell command \n")
command_response = call_tool("executeCommand",
                              {"command": "echo 'Hello World'"})
print(f"command result: {command_response}")

# Parse and display results
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

executing shell command 

command result: {
  "content": [
    {
      "type": "text",
      "text": "Hello World\r\n"
    }
  ],
  "structuredContent": {
    "stdout": "Hello World\r\n",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 0.005118608474731445
  },
  "isError": false
}
Hello World



### 8.2 Next, lets install boto3 using PIP in to the sandbox

In [17]:
# Test code execution
# S3 Operations
print("executing shell command \n")
command_response = call_tool("executeCommand",
                              {"command": "pip install boto3"})

# Parse and display results
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

executing shell command 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 3.6 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [boto3]32m3/4 [boto3]re]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip



## 9. File Operations and S3 Interaction by runninng commands

#### 9.1 Write local file to sandbox

In [ ]:
# Write file to sandbox
print("Writing file to sandbox")
try:
    with open(local_file, 'r', encoding='utf-8') as local_file_content:
        local_file_content = local_file_content.read()
except FileNotFoundError:
    print(f"Error: The file '{local_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

files_to_create = [{
        "path": "stats.py",
        "text": local_file_content
}]
write_files_response = call_tool("writeFiles", {"content": files_to_create})
print(f"write files result: {write_files_response}")

In [12]:
local_file

'samples/stats.py'

In [12]:
# Write file to sandbox
print("Writing file to sandbox")
try:
    with open("./samples/create-uv-env.sh", 'r', encoding='utf-8') as local_file_content:
        local_file_content = local_file_content.read()
except FileNotFoundError:
    print(f"Error: The file '{local_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

files_to_create = [{
        "path": "create-uv-env.sh",
        "text": local_file_content
}]
write_files_response = call_tool("writeFiles", {"content": files_to_create})
print(f"write files result: {write_files_response}")

Writing file to sandbox


KeyboardInterrupt: 

In [34]:
# Write file to sandbox
print("Writing file to sandbox")
try:
    with open("./samples/install_korean_font.sh", 'r', encoding='utf-8') as local_file_content:
        local_file_content = local_file_content.read()
except FileNotFoundError:
    print(f"Error: The file '{local_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

files_to_create = [{
        "path": "install_korean_font.sh",
        "text": local_file_content
}]
write_files_response = call_tool("writeFiles", {"content": files_to_create})
print(f"write files result: {write_files_response}")

Writing file to sandbox


ValidationException: An error occurred (ValidationException) when calling the InvokeCodeInterpreter operation: Code interpreter session 01K478RHX7Q0G8DMRJSE4E9GGV is not active

In [27]:
cmd = """
sudo apt-get update && apt-get install -y \
    pandoc \
    poppler-utils \
    fonts-nanum \
    && apt-get clean \
    && rm -rf /var/lib/apt/lists/* \
    && rm -rf /tmp/* \
    && rm -rf /var/tmp/*
"""

# Test code execution
# S3 Operations
print("executing shell command \n")
command_response = call_tool(
    "executeCommand",
    {"command": "sudo apt-get update && apt-get install -y"}
    #{"command": cmd}
    

    
)

# Parse and display results
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

executing shell command 



ReadTimeoutError: Read timeout on endpoint URL: "https://bedrock-agentcore.us-west-2.amazonaws.com/code-interpreters/s3InteractionEnv_1756890754-TCHTCJfZGz/tools/invoke"

#### 9.2 Create a S3 Bucket via code interpreter

In [18]:
# S3 Operations
print("\nCreating S3 bucket")
create_s3_response = call_tool("executeCommand",
                              {"command": f"aws s3 mb {s3_path} --region {region}"})
print(f"create result: {create_s3_response}")


Creating S3 bucket
create result: {
  "content": [
    {
      "type": "text",
      "text": "make_bucket: amzn-bucket-1756884725\r\n"
    }
  ],
  "structuredContent": {
    "stdout": "make_bucket: amzn-bucket-1756884725\r\n",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 1.2430102825164795
  },
  "isError": false
}


In [19]:
files_to_create

[{'path': 'install_korean_font.sh',
  'text': '#!/bin/bash\n\n# 스크립트 실행 시작 메시지\necho "한글 폰트 설치 및 matplotlib 설정을 시작합니다..."\n\n# 필요한 패키지 설치\necho "나눔 폰트 설치 중..."\nif command -v apt-get > /dev/null; then\n    # Ubuntu/Debian 계열\n    sudo apt-get update\n    sudo apt-get install -y fonts-nanum\nelif command -v yum > /dev/null; then\n    # CentOS/RHEL 계열\n    sudo yum install -y nanum-fonts-all\nelse\n    # 패키지 관리자를 찾을 수 없는 경우 직접 폰트 다운로드\n    echo "패키지 관리자를 찾을 수 없습니다. 직접 폰트를 다운로드합니다..."\n    mkdir -p ~/.fonts\n    cd ~/.fonts\n    if command -v wget > /dev/null; then\n        wget https://github.com/naver/nanumfont/raw/master/NanumGothic.ttf\n    elif command -v curl > /dev/null; then\n        curl -O https://github.com/naver/nanumfont/raw/master/NanumGothic.ttf\n    else\n        echo "wget 또는 curl이 설치되어 있지 않습니다. 폰트 다운로드를 건너뜁니다."\n    fi\nfi\n\n# 폰트 캐시 갱신\necho "폰트 캐시를 갱신합니다..."\nfc-cache -f -v\n\n# 설치된 폰트 확인\necho "설치된 나눔 폰트 목록:"\nfc-list | grep "Nanum"\n\n# matplotlib 설정 파일 찾기\necho "mat

#### 9.3 Upload file from code interpreter by running command to the S3 Bucket(created above)

In [20]:
print("\nUploading file to S3")
upload_to_s3_response = call_tool("executeCommand",
                                 {"command": f"aws s3 cp {files_to_create[0]['path']} {s3_path}"})
print(f"upload result: {upload_to_s3_response}")




Uploading file to S3
upload result: {
  "content": [
    {
      "type": "text",
      "text": "Completed 5.3 KiB/5.3 KiB (18.1 KiB/s) with 1 file(s) remaining\rupload: ./install_korean_font.sh to s3://amzn-bucket-1756884725/install_korean_font.sh\r\n"
    }
  ],
  "structuredContent": {
    "stdout": "Completed 5.3 KiB/5.3 KiB (18.1 KiB/s) with 1 file(s) remaining\rupload: ./install_korean_font.sh to s3://amzn-bucket-1756884725/install_korean_font.sh\r\n",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 0.8935623168945312
  },
  "isError": false
}


#### 9.4 List files from the S3 bucket by running a command in the code interpreter

In [21]:
print("\nListing files in S3")
list_s3_response = call_tool("executeCommand",
                            {"command": f"aws s3 ls {s3_path}"})
print(f"list result: {list_s3_response}")


Listing files in S3
list result: {
  "content": [
    {
      "type": "text",
      "text": "2025-09-03 07:39:40       5377 install_korean_font.sh\r\n"
    }
  ],
  "structuredContent": {
    "stdout": "2025-09-03 07:39:40       5377 install_korean_font.sh\r\n",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 0.8672411441802979
  },
  "isError": false
}


## 10. Cleanup

Clean up resources by stopping the session and deleting the interpreter.

In [ ]:
# Cleanup
print("Cleaning up session and interpreter")
dp_client.stop_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id
)
print("Session stopped successfully")

cp_client.delete_code_interpreter(codeInterpreterId=interpreter_id)
print("Interpreter deleted successfully")